In [1]:
import numpy as np
from scipy.optimize import minimize
import math
from itertools import product
#computes the entropy of a probability distribution given a list of probabilities "probabilities"
def H(probabilities):
    sum = 0.0
    for i in probabilities:
        if (i <= 0):
            sum += 0
        else:
            sum -= i * math.log2(i)
    return sum

In [2]:
H([0.1, 0.9])

0.4689955935892812

In [3]:
#computes the objective function given numpy arrays p, a, b
def loss(p, a, b):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i] * (H([a[i], 1 - a[i]]) + H([b[i], 1 - b[i]]))
    return sum

In [4]:
def h(x):
    return H([x, 1-x])

In [5]:
h(0.1)

0.4689955935892812

In [6]:
def L(x):
    return h(x) + 1 - x

In [7]:
# This computes the left hand side of the inequality in the conditions, given numpy arrays p, a, b, c (outdated)
def oldLHS(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i in range(n):
        sum1 += p[i] * (a[i] * b[i] - c[i])
        sum2 += p[i] * (a[i] * (1 - b[i]) + (1 - a[i]) * b[i] + 2 * c[i])
        sum3 += p[i] * ((1 - a[i]) * (1 - b[i]) - c[i])
    return H([sum1, sum2, sum3])

In [8]:
def LHS(p, a, b, c):
    n = len(p)
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return L(sum)

In [9]:
# This computes the right hand side of the inequality in the conditions, given numpy arrays p, a, b, c

def RHS(p, a, b, c):
    entropy_sum = 0
    n = len(p)
#     print(a)
#     print(b)
#     print(c)
    for i in range(n):
        if ((a[i] * b[i] - c[i]) < 0 or (a[i] * (1 - b[i]) + c[i]) < 0 or (b[i] * (1 - a[i]) + c[i]) < 0 or ((1 - a[i]) * (1 - b[i]) + c[i]) < 0):
            return False

    for i in range(n):
        entropy = H([a[i] * b[i] - c[i], a[i] * (1 - b[i]) + c[i], (1 - a[i]) * b[i] + c[i], (1 - a[i]) * (1 - b[i]) - c[i]])
        entropy_sum += p[i] * entropy
    return entropy_sum

In [10]:
# This computes quantity to minimize for each value of c, given numpy arrays p, a, b, c

def LHS_minus_RHS(p, a, b, c):
    if (LHS(p,a,b,c) == False or RHS(p,a,b,c) == False):
        return False
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [143]:
#Uses gradient descent to find the worst values of c possible

def worst_c(p, a, b, alpha, iterations):
    n = len(p)
    c = np.zeros(n, dtype = float)
    prevc = np.zeros(n, dtype = float)
    for i in range(iterations):
        prevc[:] = c
        for j in range(n):
            cplus = list(c)
            cplus[j] += 0.0001
            cminus = list(c)
            cminus[j] -=  0.0001
            dxdc = (LHS_minus_RHS(p, a, b, cplus) - LHS_minus_RHS(p, a, b, cminus))/.0002
            c[j] -= dxdc * alpha
        for j in range(n):
            if (a[j] * b[j] - c[j] < 0 or a[j] * (1 - b[j]) + c[j] < 0 or b[j] * (1 - a[j]) + c[j] < 0 or (1 - a[j]) * (1 - b[j]) + c[j] < 0):
                c[:] = prevc
                return c
    return c

In [12]:
def star(p,a,b,c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return sum
#LHS of the partial derivative expression
def delL(p,a,b,c):
    s = star(p,a,b,c)
    return ((1-s)/(2*s))**2

def solve_quad(a,b,c, low, high):
    s1 = (-1*b + (b**2-4*a*c)**.5)/(2*a)
    s2 = (-1*b - (b**2-4*a*c)**.5)/(2*a)
    if s1>=low and s1<=high:
        return s1
    if s2>=low and s2<=high:
        return s2
    return False

#finds worse c by solving for where partial derivative = 0 given old LHS
def update_c(s,pi,ai,bi):
    w = (1-ai)*bi
    x = ai * (1-bi)
    y = ai*bi
    z = (1-ai)*(1-bi)
    quad = s-1
    lin = s*w+s*x+y+z
    con = s*w*x - y*z
    return solve_quad(quad,lin,con, low = 0, high = 1)
def worse_c(p,a,b,c):
    s = delL(p,a,b,c)
    n = len(np.atleast_1d(p))
    c2 = np.zeros(n, dtype = float)
    for i in range(n):
        u = update_c(s,p[i],a[i],b[i])
        if u>=0 and u<=1:
            c2[i]=u
        else:
            c2[i]=c[i]
    return c2

In [13]:
a = np.array((0.23684, 0.23684, 0.23684))
b = np.array((0.23684, 0.23684, 0.23684))
c = np.array((0, 0, 0))
p = np.array((0.2, 0.8))
#starting at 0 vector and iterating a bunch could give the worst c vector
for i in range(100):
    c=worse_c(p,a,b,c)
print(c)
print(LHS_minus_RHS(p,a,b,c))

[0.01538219 0.01538219]
-8.51226210163425e-06


In [14]:
def check_c(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i]*(a[i]*(1 - b[i]) + b[i]*(1 - a[i]) + 2*c[i])
    return sum

In [15]:

def analytic_worst_c(p, a, b):
    n = len(p)
    c_worst = 0
    mindistance = 10000
    for x in  np.linspace(0.001, 1, num = 1000):
        c = []
        y = math.pow((1 - x)/(2 * x),2)
        #consider a quadratic equation pc^2+qc+r=0
        for i in range(n):
            pp = y - 1
            qq = 2*a[i]*b[i] - 2*a[i]*b[i]*y + a[i]*y + b[i]*y - a[i] - b[i] + 1
            rr = a[i]*a[i]*b[i]*b[i]*y - a[i]*a[i]*b[i]*b[i] - a[i]*a[i]*b[i]*y - a[i]*b[i]*b[i]*y + a[i]*a[i]*b[i] + a[i]*b[i]*b[i] + a[i]*b[i]*y - a[i]*b[i]
            if (solve_quad(pp, qq, rr, low = -0.25, high = 0.25) != False):
                #be careful, maybe the second root works. Need to check this never actually happens
                c.append(solve_quad(pp, qq, rr, low = -0.25, high = 0.25))
        if (len(c) != 3):
            continue
        if (abs(check_c(p, a, b, c) - x) < mindistance):
            mindistance = abs(check_c(p, a, b, c) - x)
            c_worst = c
    return c_worst

                
        
    

In [16]:
p = np.array((0.3, 0.3, 0.4))
a = np.array((0.2, 0.3, 0.7))
b = np.array((0.4, 0.2, 0.5))
print(worst_c(p, a, b, alpha = 0.001, iterations = 1000))
print(analytic_worst_c(p, a, b))

[0.05992222 0.04708042 0.09645359]
[0.05987266851545786, 0.04704648365461646, 0.09635100276050969]


In [17]:
def bsearch(a, b, low = 0, high = 0.5, eps = 0.00001):
    mid = (low + high)/2
    if ((high-low)<eps):
        return mid
    if (2*h(mid) >= (h(a)+h(b))):
        return bsearch(a, b, low, mid, eps)
    else:
        return bsearch(a, b, mid, high, eps)

In [18]:
bsearch(0.2, 0.4)

0.2734947204589844

In [19]:
h(0.2)+h(0.4)

1.692878689342031

In [20]:
2*h(0.273494)

1.6928674955011611

In [62]:
def L_minus_sum_RHS(p, a, b):
    c = worst_c(p, a, b, alpha = 0.001, iterations = 1000)
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [104]:
import random
p = [0.3, 0.3, 0.4]
mini = 100000
for i in range(1000):
    a1 = random.randint(1, 9999)/10000
    a2 = random.randint(1, 9999)/10000
    a3 = random.randint(1, 9999)/10000
    b1 = random.randint(1, 9999)/10000
    b2 = random.randint(1, 9999)/10000
    b3 = random.randint(1, 9999)/10000
    if (L_minus_sum_RHS(p,[a1,a2,a3],[b1,b2,b3])>L_minus_sum_RHS(p,[a1,a2,bsearch(a3,b3)],[b1,b2,bsearch(a3,b3)])):
        mini = min(mini, L_minus_sum_RHS(p,[a1,a2,bsearch(a3,b3)],[b1,b2,bsearch(a3,b3)]))
        break
    

In [ ]:
L_minus_sum_RHS(p,[a1,a2,a3],[b1,b2,b3])

In [ ]:
L_minus_sum_RHS(p,[a1,a2,bsearch(a3,b3, eps = 0.0000000001)],[b1,b2,bsearch(a3,b3, eps = 0.00000001)])

In [ ]:
print([a1,a2,a3])
print([b1,b2,b3])
print(bsearch(a3,b3))
print(L_minus_sum_RHS(p,[a1,a2,a3],[b1,b2,b3]))
print(L_minus_sum_RHS(p,[a1,a2,bsearch(a3,b3, eps = 0.0000000001)],[b1,b2,bsearch(a3,b3, eps = 0.00000001)]))

In [52]:
def bsearch_optimal_obj(low = 0, high = 0.5, eps = 0.0000001):
    mid = (low + high)/2
    if ((high-low)<eps):
        return[mid, 2*h(mid)]
    if (L_minus_sum_RHS([1,0],[mid,mid],[mid,mid])>0):
        return bsearch_optimal_obj(mid, high, eps)
    else:
        return bsearch_optimal_obj(low, mid, eps)

In [53]:
x = bsearch_optimal_obj()
print(x)

[0.23683777451515198, 1.579483886085085]


In [54]:
(2 * h (x[0]+0.0000001) - 2 * h(x[0]))/0.0000001

3.376180324199396

In [55]:
(L_minus_sum_RHS([0.5,0.5],[x[0]+0.0000001,x[0]+0.0000001],[x[0]+0.0000001,x[0]+0.0000001]) - L_minus_sum_RHS([0.5,0.5],[x[0],x[0]],[x[0],x[0]]))/0.0000001

-3.799937253745611

In [56]:
3.376180324199396/3.799937253745611

0.8884831771554869

In [96]:
%%time
import random
p = [0.3, 0.3, 0.4]
upper_bound = 101
lower_bound = -1
alowbest = [0,0,0]
blowbest = [0,0,0]
ahighbest = [0,0,0]
bhighbest = [0,0,0]
for i in range(100):
    if i%100 == 0:
        print(i)
        print("Upper Bound:")
        print(upper_bound)
#         print(alowbest)
#         print(blowbest)
#         print(loss(p,alowbest,blowbest))
#         print(L_minus_sum_RHS(p,alowbest,blowbest))
        print("Lower Bound:")
        print(lower_bound)
#         print(ahighbest)
#         print(bhighbest)
#         print(loss(p,ahighbest,bhighbest))
#         print(L_minus_sum_RHS(p,ahighbest,bhighbest))
        print()
        print()
    a1 = random.randint(1, 99)/100
    a2 = random.randint(1, 99)/100
    a3 = random.randint(1, 99)/100
    b1 = random.randint(1, 99)/100
    b2 = random.randint(1, 99)/100
    b3 = random.randint(1, 99)/100
    a = [a1, a2, a3]
    b = [b1, b2, b3]
    x = L_minus_sum_RHS(p,a,b)
    
    if (abs(x) < 0.2):
        continue
    loss_here = loss(p,a,b)
    if (x > 0):
        if (upper_bound>(1.5795-loss_here)/x):
            upper_bound = (1.5795-loss_here)/x
            alowbest = a
            blowbest = b
    if (x < 0):
        if (lower_bound < (1.5795-loss_here)/x):
            lower_bound = (1.5795-loss_here)/x
            ahighbest = a
            bhighbest = b    
print(upper_bound)
print(lower_bound)
        

0
Upper Bound:
101
Lower Bound:
-1


0.9738944281982432
0.6661584912122981
CPU times: user 39.5 s, sys: 140 ms, total: 39.7 s
Wall time: 40 s


In [95]:
print(maxi)
p = [0.3, 0.3, 0.4]
print(p)
print(alowbest)
print(blowbest)
print(worst_c(p,alowbest,blowbest,alpha = 0.001, iterations = 1000))

0.6451018654873548
[0.3, 0.3, 0.4]
[0.12, 0.05, 0.7]
[0.24, 0.29, 0.82]
[-1.44039746e-04 -7.31799656e-05 -2.31631122e-04]


In [88]:
print(loss(p,alowbest,blowbest))
print(L_minus_sum_RHS(p,alowbest,blowbest))

0.950377286873342
1.5003964289113132


In [89]:
c = worst_c(p,alowbest,blowbest, alpha = 0.001, iterations = 10000)
print(c)
print(LHS_minus_RHS(p,alowbest,blowbest, c))

[-0.01010377 -0.00987949 -0.01275103]
False


In [92]:
a = [.3301,.3857,0.8761]
b = [.3766,.4139,.991]
p = [0.3, 0.3, 0.4]
c = worst_c(p,a,b,alpha = 0.001, iterations = 1000)
print(LHS_minus_RHS(p,a,b,c))
print(LHS_minus_RHS(p,[.3301,.3857,bsearch(0.8761,0.991)],[.3766,.4139,bsearch(0.8761,0.991)],c))

0.19580659561702451
0.19550451117822476


In [145]:
for i in range (1000):
    a1 = random.randint(1, 99)/100
    a2 = random.randint(1, 99)/100
    a3 = random.randint(1, 99)/100
    b1 = random.randint(1, 99)/100
    b2 = random.randint(1, 99)/100
    b3 = random.randint(1, 99)/100
    a = [a1,a2,a3]
    b = [b1,b2,b3]
    p = [0.3,0.3,0.4]
    c = worst_c(p,a,b, alpha = 0.001, iterations = 100)
    if (LHS_minus_RHS(p,a,b,c) == False):
        print(p)
        print(a)
        print(b)
        print(c)
        print(loss(p,a,b))
        print()
        break

In [144]:
c=worst_c(p,a,b, alpha = 0.001, iterations = 100)
print(c)
LHS_minus_RHS(p,a,b,c)

[-0.0159879  -0.00525231 -0.00891165]


0.4083863129993883